In [1]:
pip install -q -U google-genai

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import gradio
import openai
import anthropic
from google import genai
from google.genai import types
from dotenv import load_dotenv

In [2]:
load_dotenv()
gpt = openai.OpenAI()
claude = anthropic.Anthropic()
gemini = genai.Client(api_key=os.getenv('GOOGLE_API_KEY'))



In [3]:
def func(input):
    print(f"Get input = {input}")
    return input.upper()

In [35]:
systemMessage = 'You are a very helpful assistant, specializing in translations. Respond in plain text.'

In [36]:
def ask_gpt(prompt):
    completion = gpt.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {'role': 'system', 'content': systemMessage},
            {'role': 'user', 'content': prompt}
        ]
    )

    return completion.choices[0].message.content

In [37]:
def stream_gpt(prompt):
    stream = gpt.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {'role': 'system', 'content': systemMessage},
            {'role': 'user', 'content': prompt}
        ],
        stream=True
    )
    print(type(stream))
    result = ''
    for chunk in stream:
        result += chunk.choices[0].delta.content or ''
        yield result

In [38]:
def ask_claude(prompt):
    response = claude.messages.create(
        model='claude-3-haiku-20240307',
        max_tokens=100,
        system=systemMessage,
        messages=[
            {'role': 'user', 'content': prompt}
        ]
    )
    return response.content[0].text

In [39]:
def stream_claude(prompt):
    response = claude.messages.stream(
        model='claude-3-haiku-20240307',
        max_tokens=100,
        system=systemMessage,
        messages=[
            {'role': 'user', 'content': prompt}
        ]
    )
    print(type(response))
    result = ''
    with response as stream:
        for text in stream.text_stream:
            result += text or ''
            yield result

In [40]:
def ask_gemini(prompt):
    response = gemini.models.generate_content(
        model='gemini-2.0-flash',
        config=types.GenerateContentConfig(
            system_instruction=systemMessage),
        contents=prompt
    )
    return response.text

In [41]:
def stream_gemini(prompt):
    response = gemini.models.generate_content_stream(
        model='gemini-2.0-flash',
        config=types.GenerateContentConfig(
            system_instruction=systemMessage),
        contents=prompt
    )
    print(type(response))
    result = ''
    for chunk in response:
        result += chunk.text or ''
        yield result

In [42]:
def ask_ai(prompt, model):
    if model == 'GPT':
        yield ask_gpt(prompt)
    elif model == 'Claude':
        yield ask_claude(prompt)
    elif model == 'Gemini':
        yield ask_gemini(prompt)
    elif model == 'GPT Stream':
        yield from stream_gpt(prompt)
    elif model == 'Claude Stream':
        yield from stream_claude(prompt)
    elif model == 'Gemini Stream':
        yield from stream_gemini(prompt)
    else:
        raise Exception('No woman, no cry')
        

In [43]:
view = gradio.Interface(
    fn = ask_ai,
    inputs = [gradio.Textbox(value='переведи на русский: '), 
              gradio.Dropdown(['Gemini Stream', 'GPT Stream', 'Claude Stream'], 
                              label='Model')],
    outputs = "textbox",
    flagging_mode = 'never'
)

In [44]:
view.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


<class 'generator'>
